In [7]:
from crewai import Agent, Task, Process, Crew
from langchain_groq import ChatGroq
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.prompts import PromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun

In [8]:
from utils import get_llm_api_key

GROQ_API_KEY = get_llm_api_key("GROQ")
GROQ_MODEL_NAME = "mixtral-8x7b-32768"

groq_llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name=GROQ_MODEL_NAME)



In [9]:
template = """This is a conversation between a human and and AI agent:
{chat_history}
write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)

summary_chain = LLMChain(
    llm = groq_llm,
    prompt = prompt,
    verbose = True,
    memory=readonlymemory
)

In [10]:
# Create Seach Tool
search_tool = DuckDuckGoSearchRun()

tool_use = [
    Tool(
        name="Search",
        func=search_tool.run,
        description="Useful for searching the web for information about current Event"
    ),
    Tool(
        name="Summarize",
        func=summary_chain.run,
        description="Useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
    )
]

tool_use_1 = [
    Tool(
        name="Summarize",
        func=summary_chain.run,
        description="Useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary."
    )
]

In [11]:
# Define Agent
email_author = Agent(
    role="Professional Email Author",
    goal="Craft concise and engaging emails",
    backstory="Experience in writing impactful marketing email",
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
    tools=tool_use
)

marketing_strategist = Agent(
    role="Marketing Strategist",
    goal="Lead the marketing team in creating effective cold emails",
    backstory="A seasoned Chief Marketing Officer with a Keen eye for standout marketing content",
    verbose=True,
    allow_delegation=True,
    llm=groq_llm,
    tools=tool_use_1
)

content_specialist = Agent(
    role="Content Specialist",
    goal="Critique and refine email content",
    backstory="A professional copywriter with a wealth of experience in persuasive writing",
    verbose=True,
    allow_delegation=False,
    llm=groq_llm,
    tools=tool_use_1
)

In [12]:
# Define Task
email_task = Task(
    description=(
        """
        1. Generate two distinct variations of a cold email prompting a video solution.
        2. Evaluate the written emails for effectiveness and engagement.
        3. Scrutinize the emails for grammatical errors and clarity.
        4. Adjust the emails to align with best practices for cold outreach. Consider the feedback provided to the marketing strategist.
        5. Revise the emails based on all feedback, creating two final versions.
        """
    ),
    expected_output=(
        """
        two final versions of a cold email in json format
        """
    ),
    agent=marketing_strategist,
    max_iter=2
)

In [13]:
# Create a single Crew
email_crew = Crew(
    agents=[email_author, marketing_strategist, content_specialist],
    tasks=[email_task],
    verbose=True,
    process=Process.sequential
)

In [14]:
# Execution flow
print("Crew: Working on Email Tasks")
emails_output = email_crew.kickoff()

Crew: Working on Email Tasks
 [DEBUG]: == Working Agent: Marketing Strategist
 [INFO]: == Starting Task: 
        1. Generate two distinct variations of a cold email prompting a video solution.
        2. Evaluate the written emails for effectiveness and engagement.
        3. Scrutinize the emails for grammatical errors and clarity.
        4. Adjust the emails to align with best practices for cold outreach. Consider the feedback provided to the marketing strategist.
        5. Revise the emails based on all feedback, creating two final versions.
        


> Entering new CrewAgentExecutor chain...
Thought: I need to generate two distinct variations of a cold email prompting a video solution. I will delegate the task of writing the first email to the Professional Email Author and write the second email myself.

Action: Delegate work to co-worker
Action Input: {"coworker": "Professional Email Author", "task": "Write a cold email promoting our video solution", "context": "The email shou

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("Obama's first name?")

"Barack Obama—with his wife, Michelle—being sworn in as the 44th president of the United States, January 20, 2009. Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Most common names of U.S. presidents 1789-2021. Published by. Aaron O'Neill , Feb 2, 2024. The most common first name for a U.S. president is James, followed by John and then William. Six U.S ... Michelle Obama (born January 17, 1964, Chicago, Illinois, U.S.) is an American first lady (2009-17), the wife of Barack Obama, 44th president of the United States.She was the first African American first lady. Michelle Robinson, who grew up on Chicago's South Side, was the daughter of Marian, a homemaker, and Frasier Robinson, a worker in the city's water-purification plant. Barack and Michelle Obama's daughters, Malia and Sasha, grew up in the White House from 2009 to 2017. To mos

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
result = search.run("Obama")

In [ ]:
type(result)

str

In [ ]:
result

"[snippet: Barack Obama—with his wife, Michelle—being sworn in as the 44th president of the United States, January 20, 2009. Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office., title: Barack Obama | Biography, Parents, Education, Presidency, Books ..., link: https://www.britannica.com/biography/Barack-Obama], [snippet: 1:12. Former President Barack Obama, in a CNN interview that aired Thursday night, said he does not believe President Joe Biden will face a serious primary challenge during his 2024 reelection ..., title: Five takeaways from Barack Obama's CNN interview on Biden, Trump, 2024, link: https://www.usatoday.com/story/news/politics/2023/06/23/five-takeaways-from-barack-obama-cnn-interview/70349112007/], [snippet: In private meeting, Obama told Biden he would do whatever it takes to help in 2024. The former president made the co